In [1]:
# data analysis, wrangling and preprocessing
import numpy as np 
import pandas as pd

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
plt.style.use('fivethirtyeight') # change the style of the plot

# time series analysis
from pandas import datetime
from statsmodels.tsa.stattools import pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm

# preprocessing 
from sklearn.preprocessing import MinMaxScaler

# time series forecasting 
from pmdarima.arima import ndiffs
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pmdarima as pm

# model evaluation 
import math
from sklearn.metrics import mean_squared_error

<ipython-input-1-88247441de78>:12: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime


### Import Dataset

In [2]:
dataset = pd.read_csv('../dataset.csv')
dataset.head()
dataset.index = pd.to_datetime(dataset.index)
dataset.index.freq = 'MS'

In [3]:
dataset['SP500 Close Lag 1'] = dataset['SP500 Close'].shift(-1)
dataset.head()

,CPIAUCSL_PC1,UNRATE,FEDFUNDS,T10Y2Y,High,Low,SP500 Close,Volume,ADX,RSI,BB width,SP500 Close Lag 1
1976-06-01,5.98131,7.6,5.48,0.801364,102.507728,100.982727,101.766818,1.896500e+07,NaN,NaN,NaN,104.200953
1976-07-01,5.55556,7.8,5.31,0.981905,104.933810,103.501905,104.200953,1.875048e+07,NaN,NaN,NaN,103.291818
1976-08-01,5.71956,7.8,5.29,1.137273,103.985909,102.575001,103.291818,1.575818e+07,NaN,NaN,NaN,105.453333
1976-09-01,5.49451,7.6,5.25,1.178095,106.130951,104.637618,105.453333,1.975000e+07,NaN,NaN,NaN,101.885238
1976-10-01,5.46448,7.7,5.02,1.427000,102.681905,101.036191,101.885238,1.720905e+07,NaN,NaN,NaN,101.193500


In [4]:
dataset.tail()

,CPIAUCSL_PC1,UNRATE,FEDFUNDS,T10Y2Y,High,Low,SP500 Close,Volume,ADX,RSI,BB width,SP500 Close Lag 1
2021-12-01,7.09654,3.9,0.08,0.785000,4701.581854,4642.169611,4674.772772,3.122720e+09,45.523755,85.192588,0.543355,4573.815454
2022-01-01,7.52593,4.0,0.08,0.783500,4619.576025,4528.042017,4573.815454,3.663972e+09,46.122325,78.669780,0.501704,4435.980494
2022-02-01,7.91202,3.8,0.08,0.498947,4473.607345,4392.072625,4435.980494,3.850936e+09,45.583801,70.709690,0.460878,4391.265264
2022-03-01,8.55759,3.6,0.20,0.218261,4424.881645,4351.570015,4391.265264,4.390362e+09,44.777416,68.295438,0.419147,4468.859945
2022-04-01,NaN,NaN,NaN,0.155556,4500.185872,4439.167440,4468.859945,3.565105e+09,44.302466,70.197059,0.389122,NaN


In [5]:
model = ARIMA(dataset['SP500 Close Lag 1'].dropna(), order=(0,1,1)) 
model_fit = model.fit()

In [6]:
forecast = model_fit.forecast(steps=1)
print(forecast)
forecast_ci = model_fit.get_forecast(steps=1)
ci = forecast_ci.conf_int(alpha=0.05)
print(ci)

2022-04-01    4484.348629
Freq: MS, dtype: float64
            lower SP500 Close Lag 1  upper SP500 Close Lag 1
2022-04-01              4384.229445              4584.467814


In [11]:
#### forecast = forecast[0]
monthly_return = forecast / dataset['SP500 Close Lag 1'].dropna()[-1] - 1
monthly_return[0]

0.003465914124217262

In [8]:
# function that's used to annualize a monthly return 
def annualize(monthly):
    return round((1 + monthly) ** 12 - 1, 4)

In [10]:
annualize(monthly_return)[0]

0.0424